**Install packages**

In [33]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Setup Firebase connection**

In [34]:
from firebase import firebase

# Firebase connection setup
FBconn = firebase.FirebaseApplication('https://cloudtirgul5-default-rtdb.firebaseio.com/', None)

**Login Page**

In [35]:
def display_login_menu():
    clear_output(wait=True)

    # HTML FORMAT
    html_code = """
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">
    <div class="container d-flex justify-content-center align-items-center">
        <div class="text-center">
            <p class="display-4" style="color: black;">Cloud Battleship Trivia</p>
            <img src="https://staticctf.ubisoft.com/J3yJr34U2pZ2Ieem48Dwy9uqj5PNUQTn/5UCI3AbvqSJ8QCnNKdGFfs/7ba3a4d1ef931142b5546bb2c13d26d1/-BS-_Feature_-_4.jpg" style="width: 600px; height: 400px;" class="img-fluid" alt="Responsive image">
            <p class="display-4 mt-3" style="font-size: 36px">Login</p>
            <p class="display-4 mt-3" style="font-size: 20px">Welcome to Cloud Battleship Trivia! In order to play, please login.</p>
            <p class="display-4" style="font-size: 18px">New player? Register now!</p>
            <div class="form-group container d-flex justify-content-center align-items-center">
                <label for="usernameInput" class="mt-3 mr-2"><strong>Username:</strong></label>
                <input type="text" class="form-control mt-2" id="usernameInput" placeholder="Please enter username...">
            </div>
            <div class="form-group container d-flex justify-content-center align-items-center">
                <label for="passwordInput" class="mt-3 mr-2"><strong>Password:</strong></label>
                <input type="password" class="form-control mt-2" id="passwordInput" placeholder="Please enter password...">
            </div>
            <button class="btn btn-success mt-2" style="width: 200px" onclick="login()">Login</button>
            <button class="btn btn-primary mt-2 ml-2" style="width: 200px" onclick="register()">Register</button>
            <div id="errorMessage" class="mt-2"></div>
            </div>
        </div>
    </div>

    <!-- JavaScript link for Bootstrap -->
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>

    <script>
        async function register() {
            var usernameInput = document.getElementById('usernameInput');
            var passwordInput = document.getElementById('passwordInput');
            var errorMessage = document.getElementById('errorMessage');

            usernameInput.classList.remove("is-invalid", "is-valid");
            passwordInput.classList.remove("is-invalid", "is-valid");
            errorMessage.innerHTML = "";

            if (usernameInput.value === "" || passwordInput.value === "") {
                if (usernameInput.value === "") {
                    usernameInput.classList.add("is-invalid");
                } else {
                    usernameInput.classList.add("is-valid");
                }
                if (passwordInput.value === "") {
                    passwordInput.classList.add("is-invalid");
                } else {
                    passwordInput.classList.add("is-valid");
                }
                errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>All fields are required.</div>";
            } else {
                var userExistsResponse = await google.colab.kernel.invokeFunction("user_exists", [usernameInput.value]);
                var userExists = userExistsResponse.data['text/plain'];
                if (userExists == "True") {
                    errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>User already exists!</div>";
                } else {
                    await google.colab.kernel.invokeFunction("register_user", [usernameInput.value, passwordInput.value]);
                    usernameInput.value = "";
                    passwordInput.value = "";
                    errorMessage.innerHTML = "<div class='alert alert-success' role='alert'>Registration successful.</div>";
                }
            }
        }



        async function login() {
            var usernameInput = document.getElementById('usernameInput');
            var passwordInput = document.getElementById('passwordInput');
            var errorMessage = document.getElementById('errorMessage');

            usernameInput.classList.remove("is-invalid", "is-valid");
            passwordInput.classList.remove("is-invalid", "is-valid");
            errorMessage.innerHTML = "";

            if (usernameInput.value === "" || passwordInput.value === "") {
                if (usernameInput.value === "") {
                    usernameInput.classList.add("is-invalid");
                } else {
                    usernameInput.classList.add("is-valid");
                }
                if (passwordInput.value === "") {
                    passwordInput.classList.add("is-invalid");
                } else {
                    passwordInput.classList.add("is-valid");
                }
                errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>All fields are required.</div>";
            } else {
                await google.colab.kernel.invokeFunction("check_login", [usernameInput.value, passwordInput.value]);
                usernameInput.classList.add("is-invalid");
                passwordInput.classList.add("is-invalid");
                errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>The username or password are incorrect.</div>";
            }
        }
    </script>
    """

    display(HTML(html_code))


In [36]:
from IPython.display import display, HTML

def check_login(username, password):

    global current_user_id
    # Fetch all users from the database
    users = FBconn.get('/users', None)

    # Check if the username and password match any user in the database
    for user_key, user_data in users.items():
        if user_data['username'] == username and user_data['password'] == password:
            current_user_id = user_key
            display_main_menu()
            return # exit the function if a match is found

In [37]:
# function to check if the user exists
def user_exists(username):
    users = FBconn.get('/users', None)

    if users is not None:
        for key, val in users.items():
            if val['username'] == username:
                return True

    return False


def register_user(username, password):
    new_user = {
        'username': username,
        'password': password,
        'total_questions': 0,
        'total_correct_answers': 0,
        'total_wins': 0
    }
    FBconn.post('/users', new_user)



**Main Menu**

In [76]:
from IPython.display import display, clear_output, HTML
from google.colab import output

# Function to display the main menu
def display_main_menu():
    clear_output(wait=True)

    # HTML FORMAT
    html_code = """
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">
    <div class="container d-flex justify-content-center align-items-center background">
        <div class="text-center">
            <p class="display-4" style="color: black;">Cloud Battleship Trivia</p>
            <img src="https://staticctf.ubisoft.com/J3yJr34U2pZ2Ieem48Dwy9uqj5PNUQTn/5UCI3AbvqSJ8QCnNKdGFfs/7ba3a4d1ef931142b5546bb2c13d26d1/-BS-_Feature_-_4.jpg" style="width: 600px; height: 400px;" class="img-fluid" alt="Responsive image">
            <p class="display-4 mt-3" style="font-size: 36px">Main Menu</p>

            <!-- Bootstrap Buttons -->
            <div class="btn-group mt-3" role="group" aria-label="Buttons">
                <button class="btn btn-primary rounded" style="width: 200px" onclick="adminPage()">Admin Page</button>
                <button class="btn btn-primary ml-2 rounded" style="width: 200px" onclick="startGame()">Start Game</button>
                <button class="btn btn-primary ml-2 rounded" style="width: 200px" onclick="displayRules()">Rules</button>
                <button class="btn btn-primary ml-2 rounded" style="width: 200px" onclick="displayLeaderboard()">Leaderboard</button>
                <button class="btn btn-danger ml-2 rounded" style="width: 200px" onclick="logoutAndMoveToLoginPage()">Logout</button>
            </div>
        </div>
    </div>

    <!-- JavaScript link for Bootstrap -->
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>


    <script>

        // Button click event handlers
        function adminPage() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("display_admin_login");
        }

        function startGame() {
            // Trigger the start_game_button lambda
            google.colab.kernel.invokeFunction("display_ship_placement");
        }

        function displayRules() {
            // Trigger the display_rules lambda
            google.colab.kernel.invokeFunction("display_rules");
        }

        function displayLeaderboard() {
            // Trigger the displayLeaderboard lambda
            google.colab.kernel.invokeFunction("display_leaderboard");
        }

        function logoutAndMoveToLoginPage(){
            // Trigger the logoutAndMoveToLoginPage lambda
            google.colab.kernel.invokeFunction("display_login_menu");
        }
    </script>
    """

    display(HTML(html_code))

**Rules**

In [39]:
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

def display_rules():
    clear_output(wait=True)

    rules_html = '''
    <link href=https://cdn.jsdelivr.net/npm/bootstrap@5.1.1/dist/css/bootstrap.min.css rel="stylesheet" integrity="sha384-F3w7mX95PdgyTmZZMECAngseQB83DfGTowi0iMjiWaeVhAn4FJkqJByhZMI3AhiU" crossorigin="anonymous">
    <div class="text-center" style="max-width: 700px; margin: 0 auto;">
          <p class="display-4" style="color: black;">Cloud Battleship Trivia</p>
          <img src="https://staticctf.ubisoft.com/J3yJr34U2pZ2Ieem48Dwy9uqj5PNUQTn/5UCI3AbvqSJ8QCnNKdGFfs/7ba3a4d1ef931142b5546bb2c13d26d1/-BS-_Feature_-_4.jpg" style="width: 600px; height: 400px;" class="img-fluid" alt="Responsive image">
          <p class="display-4 mt-3" style="font-size: 36px">Rules</p>
          <p class="display-4" style="font-size: 18px">The rules of the game are described below.</p>
          <p class="display-4" style="font-size: 18px">Please read them carefully and play accordingly.</p>

          <div class="accordion" id="accordionExample">
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingOne">
                <button class="accordion-button" type="button" data-bs-toggle="collapse" data-bs-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
                  Ship Placement
                </button>
              </h2>
              <div id="collapseOne" class="accordion-collapse collapse show" aria-labelledby="headingOne" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>At the start of the game, you'll need to place your ships on your grid. This is done by selecting the grid coordinates where you want each of your ships to be located. Each ship occupies a certain number of adjacent grid spaces, either in a row or a column.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingTwo">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseTwo" aria-expanded="false" aria-controls="collapseTwo">
                  Start the Game
                </button>
              </h2>
              <div id="collapseTwo" class="accordion-collapse collapse" aria-labelledby="headingTwo" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>After your ships are placed, the game starts by presenting you with a trivia question. The game continues with alternating turns of trivia and battleship moves.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingThree">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseThree" aria-expanded="false" aria-controls="collapseThree">
                  Answering Trivia Questions
                </button>
              </h2>
              <div id="collapseThree" class="accordion-collapse collapse" aria-labelledby="headingThree" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>You'll be presented with a trivia question. If you answer correctly, you'll get a chance to make a move in the battleship game and move to the next question untill your answer is incorrect.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingFour">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseFour" aria-expanded="false" aria-controls="collapseFour">
                  Making a Move in Battleship
                </button>
              </h2>
              <div id="collapseFour" class="accordion-collapse collapse" aria-labelledby="headingFour" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>If your answer to the trivia question is correct, you get to attack your opponent's board. You select a grid coordinate to attack.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingFive">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseFive" aria-expanded="false" aria-controls="collapseFive">
                  Opponent's Turn
                </button>
              </h2>
              <div id="collapseFive" class="accordion-collapse collapse" aria-labelledby="headingFive" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>After your answer is incorrect, the computer opponent makes a move, attacking your board.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingSix">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseSix" aria-expanded="false" aria-controls="collapseSix">
                  Checking Game Progress
                </button>
              </h2>
              <div id="collapseSix" class="accordion-collapse collapse" aria-labelledby="headingSix" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>At any point during the game, you can check your statistics or view your own board to see the computer's attacks by clicking the buttons "Show Statistic" or "Show my board".</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingSeven">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseSeven" aria-expanded="false" aria-controls="collapseSeven">
                  Ending the Game
                </button>
              </h2>
              <div id="collapseSeven" class="accordion-collapse collapse" aria-labelledby="headingSeven" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>The game ends when all the ships on one player's board have been sunk. The player with remaining ships is declared the winner.</strong>
                </div>
              </div>
            </div>
            <div class="accordion-item">
              <h2 class="accordion-header" id="headingEight">
                <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#collapseEight" aria-expanded="false" aria-controls="collapseEight">
                  Exiting the Game
                </button>
              </h2>
              <div id="collapseEight" class="accordion-collapse collapse" aria-labelledby="headingEight" data-bs-parent="#accordionExample">
                <div class="accordion-body">
                  <strong>You can choose to exit the game at any point by clicking the "Exit Game" button. When exiting, the game will show your correct and incorrect answers as a pie chart, and reset the game for a new session.</strong>
                </div>
              </div>
            </div>
          </div>

        <!-- Bootstrap Buttons -->
        <div class="btn-group mt-3" role="group" aria-label="Buttons">
        <button class="btn btn-primary rounded" style="width: 200px" onclick="backToMainMenu()">Back</button>
        </div>
    </div>

    <script>
        // Button click event handlers
        function backToMainMenu() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("on_back_click");
        }
    </script>

    <script src=https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js referrerpolicy="no-referrer"></script>
    <script src=https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.3/dist/umd/popper.min.js integrity="sha384-W8fXfP3gkOKtndU4JGtKDvXbO53Wy8SZCQHczT5FMiiqmQfUpWbYdTil/SxwZgAN" crossorigin="anonymous"></script>
    <script src=https://cdn.jsdelivr.net/npm/bootstrap@5.1.1/dist/js/bootstrap.min.js integrity="sha384-skAcpIdS7UcVUC05LJ9Dxay8AXcDYfBJqt1CJ85S/CFujBsIzCIv+l9liuYLaMQ/" crossorigin="anonymous"></script>
    '''

    display(HTML(rules_html))

    back_button = widgets.Button(description="Back", button_style='info', layout=widgets.Layout(width='auto'))
    display(HTML("<div style='text-align: center;'>"))
    display(HTML("</div>"))

def on_back_click():
  display_main_menu()


**Admin**

In [74]:
def display_admin_login():

    clear_output(wait=True)

    # HTML FORMAT
    html_code = """
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">
    <div class="container d-flex justify-content-center align-items-center">
        <div class="text-center">
            <p class="display-4" style="color: black;">Cloud Battleship Trivia</p>
            <img src="https://staticctf.ubisoft.com/J3yJr34U2pZ2Ieem48Dwy9uqj5PNUQTn/5UCI3AbvqSJ8QCnNKdGFfs/7ba3a4d1ef931142b5546bb2c13d26d1/-BS-_Feature_-_4.jpg" style="width: 600px; height: 400px;" class="img-fluid" alt="Responsive image">
            <p class="display-4 mt-3" style="font-size: 36px">Admin Page</p>
            <p class="display-4 mt-3" style="font-size: 20px">Please enter the admin password in order to enter the Admin Page.</p>
            <div class="form-group container d-flex justify-content-center align-items-center">
                <label for="passwordInput" class="mt-3 mr-2"><strong>Password:</strong></label>
                <input type="password" class="form-control mt-2" id="passwordInput" placeholder="Please enter admin password...">
            </div>
            <button class="btn btn-success mt-2" style="width: 200px" onclick="enterAdminPage()">Submit</button>
            <button class="btn btn-primary mt-2 ml-2" style="width: 200px" onclick="backToMainMenu()">Back</button>
            <div id="errorMessage" class="mt-2"></div>
            </div>
        </div>
    </div>

    <!-- JavaScript link for Bootstrap -->
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>


    <script>
        function enterAdminPage() {
            var passwordInput = document.getElementById('passwordInput');
            var errorMessage = document.getElementById('errorMessage');

            passwordInput.classList.remove("is-invalid", "is-valid");
            errorMessage.innerHTML = "";

            if (passwordInput.value === "") {
                passwordInput.classList.add("is-invalid");
                errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>Password is required in order to enter Admin Page.</div>";
            } else {
                google.colab.kernel.invokeFunction('on_submit_click', [passwordInput.value]).then(function(result) {
                  console.log(result)
                    if (result.data['text/plain'] === "'fail'") {
                        passwordInput.classList.add("is-invalid");
                        errorMessage.innerHTML = "<div class='alert alert-danger' role='alert'>Password is incorrect.</div>";
                    } else {
                        // Handle successful login here, e.g., navigate to admin page
                    }
                });
            }
        }

        // Button click event handlers
        function backToMainMenu() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("on_back_click");
        }
    </script>
    """

    display(HTML(html_code))

def on_submit_click(password):
    # Fetch admin password from the database
    if password == str(FBconn.get('/admin', 'password')):
        admin_options()
        return "success" # Return success if the password is correct
    else:
        return "fail" # Return fail if the password is incorrect

def on_back_click():
    display_main_menu()

In [41]:
def admin_options():

    clear_output(wait=True)

    # HTML FORMAT
    html_code = """
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">
    <div class="container d-flex justify-content-center align-items-center">
        <div class="text-center">
            <p class="display-4" style="color: black;">Cloud Battleship Trivia</p>
            <img src="https://staticctf.ubisoft.com/J3yJr34U2pZ2Ieem48Dwy9uqj5PNUQTn/5UCI3AbvqSJ8QCnNKdGFfs/7ba3a4d1ef931142b5546bb2c13d26d1/-BS-_Feature_-_4.jpg" style="width: 600px; height: 400px;" class="img-fluid" alt="Responsive image">
            <p class="display-4 mt-3" style="font-size: 36px">Admin Page</p>
            <p class="display-4 mt-3" style="font-size: 20px">Hello admin, what would you like to do?</p>
            <button class="btn btn-success mt-2" style="width: 200px" onclick="addNewQuestion()">Add New Question</button>
            <button class="btn btn-success mt-2 ml-2" style="width: 200px" onclick="editQuestions()">Edit Questions</button>
            <button class="btn btn-primary mt-2 ml-2" style="width: 200px" onclick="backToMainMenu()">Back</button>

            <div id="errorMessage" class="mt-2"></div>
            </div>
        </div>
    </div>

    <!-- JavaScript link for Bootstrap -->
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>


    <script>
        // Button click event handlers

        function addNewQuestion() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("on_add_question_click");
        }

        function editQuestions() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("on_edit_question_click");
        }

        function backToMainMenu() {
            // Trigger the add_question_button lambda
            google.colab.kernel.invokeFunction("on_back_click");
        }
    </script>
    """

    display(HTML(html_code))

def on_add_question_click():
    admin_input()

def on_edit_question_click():
    edit_questions() # You need to implement this function

def on_back_click():
    display_main_menu()

In [42]:
def admin_input():
    clear_output(wait=True)

    display(HTML("<h3 style='text-align: center;'>Enter the question and answers:</h3>"))
    question_input = widgets.Text(description="Question", layout=widgets.Layout(width="80%"))
    answer_inputs = [widgets.Text(description=f"Option {i + 1}", layout=widgets.Layout(width="80%")) for i in range(4)]
    correct_dropdown = widgets.Dropdown(options=[1, 2, 3, 4], description="Correct", layout=widgets.Layout(width="80%"))
    submit_button = widgets.Button(description="Submit", button_style='success', layout=widgets.Layout(width="10%", margin="10px"))
    cancel_button = widgets.Button(description="Cancel", button_style='danger', layout=widgets.Layout(width="10%", margin="10px"))

    display(question_input)
    for answer_input in answer_inputs:
        display(answer_input)
    display(correct_dropdown)

    button_container = widgets.VBox([submit_button, cancel_button], layout=widgets.Layout(align_items="center"))
    display(button_container)

    def on_submit_click(button):
        question = question_input.value
        answers = [answer_input.value for answer_input in answer_inputs]

        if "" in answers or question == "":
            display(HTML("<p style='color: red; text-align: center;'>Please fill in all fields.</p>"))
            return

        correct = correct_dropdown.value
        add_question(question, correct, *answers)
        display_main_menu()

    def on_cancel_click(button):
        admin_options()

    submit_button.on_click(on_submit_click)
    cancel_button.on_click(on_cancel_click)

In [43]:
def edit_questions():
    clear_output(wait=True)

    display(HTML("<h2 style='text-align: center;'>Edit Questions</h2>"))
    question_data = FBconn.get('/questions', None)

    question_list = widgets.Select(
        options=[(q_data["Question"], (q_key, q_data)) for q_key, q_data in question_data.items()],
        layout=widgets.Layout(width="40%", height="200px")
    )

    edit_button = widgets.Button(
        description="Edit",
        button_style='info',
        layout=widgets.Layout(width="150px", height="40px")
    )
    back_button = widgets.Button(
        description="Back",
        button_style='danger',
        layout=widgets.Layout(width="150px", height="40px")
    )

    display(question_list)
    display(widgets.HBox([edit_button, back_button], layout=widgets.Layout(justify_content="center", margin="15px")))

    def on_edit_click(button):
       question_key, question_data = question_list.value
       edit_question(question_key, question_data)

    def on_back_click(button):
        admin_options()

    edit_button.on_click(on_edit_click)
    back_button.on_click(on_back_click)


In [44]:
def list_questions():
    questions = FBconn.get('/questions', None)
    question_list = [(value['Question'], (key, value)) for key, value in questions.items()]
    return question_list

In [45]:
# This function used to add a question to the database
def add_question(question, correct, *answers):
    question_data = {
        "Question": question,
        "Option1": answers[0],
        "Option2": answers[1],
        "Option3": answers[2],
        "Option4": answers[3],
        "Correct": correct
    }
    FBconn.post('/questions', question_data)


# This function used to edit a question in the database
def edit_question(question_key, question_data):
    clear_output(wait=True)

    display(HTML("<h3 style='text-align: center;'>Edit the question and answers:</h3>"))

    question_input = widgets.Text(description="Question", value=question_data['Question'], layout=widgets.Layout(width="80%"))
    answer_inputs = [widgets.Text(description=f"Option {i + 1}", value=question_data[f"Option{i + 1}"], layout=widgets.Layout(width="80%")) for i in range(4)]
    correct_dropdown = widgets.Dropdown(options=[1, 2, 3, 4], description="Correct", value=question_data['Correct'], layout=widgets.Layout(width="80%"))
    update_button = widgets.Button(description="Update", button_style='success', layout=widgets.Layout(width="10%", margin="10px"))
    cancel_button = widgets.Button(description="Cancel", button_style='danger', layout=widgets.Layout(width="10%", margin="10px"))

    display(question_input)
    for answer_input in answer_inputs:
        display(answer_input)
    display(correct_dropdown)

    button_container = widgets.VBox([update_button, cancel_button], layout=widgets.Layout(align_items="center"))
    display(button_container)

    def on_update_click(button):
        question = question_input.value
        answers = [answer_input.value for answer_input in answer_inputs]
        if "" in answers or question == "":
            display(HTML("<p style='color: red; text-align: center;'>Please fill in all fields.</p>"))
            return
        correct = correct_dropdown.value
        question_data = {
            "Question": question,
            "Option1": answers[0],
            "Option2": answers[1],
            "Option3": answers[2],
            "Option4": answers[3],
            "Correct": correct
        }
        FBconn.put('/questions', question_key, question_data)  # replace existing question with new data
        edit_questions()  # display the updated list of questions


    def on_cancel_click(button):
        edit_questions()  # display the list of questions without making changes

    update_button.on_click(on_update_click)
    cancel_button.on_click(on_cancel_click)

**Start Game**

In [46]:
from ipywidgets.widgets.widget_style import Style
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time

def display_title_and_instructions():
    title = widgets.HTML("<h1 style='text-align: center;'>Place Your Ships</h1>")
    instructions = widgets.HTML("<h3 style='text-align: center;'>Click on a ship and then click on the grid to place it.</h3>")
    display(widgets.VBox([title, instructions]))
    return title, instructions

def create_grid_buttons(computer_board, user_board):
    grid_buttons = []
    for i in range(10):
        row = []
        for j in range(10):
            button = widgets.Button(
                layout=widgets.Layout(width='50px', height='50px'),
                style=widgets.ButtonStyle(button_color='lightblue')
            )
            button.row, button.col = i, j
            button.on_click(lambda button, computer_board=computer_board: on_attack_click(computer_board, button, grid_buttons, user_board))
            row.append(button)
        grid_buttons.append(row)
    return grid_buttons

def create_grid_buttons_start():
    grid_buttons = [
        [widgets.Button(description="",style=widgets.ButtonStyle(button_color='lightblue'), layout=widgets.Layout(width="50px", height="50px"),
                        _dom_classes=["grid-button"]) for _ in range(10)]
        for _ in range(10)
    ]
    return grid_buttons

def create_ship_buttons(ship_list):
    # Button style
    button_style = widgets.ButtonStyle(
        button_color="lightblue",
        font_weight="bold",
        border_radius="12px"
    )

    ship_buttons = [
        widgets.Button(description=f"{ship['name']} ({ship['size']})",style=button_style, layout=widgets.Layout(width="180px"))
        for ship in ship_list
    ]
    return ship_buttons

def create_grid_and_ships_layout(grid_buttons, ship_buttons, title, instructions, orientation_button, progress_bar, error_msg, start_game_button):
    grid_box = widgets.GridBox(
        [button for row in grid_buttons for button in row],
        layout=widgets.Layout(grid_template_columns="repeat(10, 50px)")
    )

    start_game_button.layout.visibility = 'hidden'
    display(widgets.VBox([
        widgets.HBox([
            widgets.VBox(ship_buttons + [widgets.Box(layout=widgets.Layout(height="15px")), orientation_button, progress_bar, error_msg, start_game_button]),
            grid_box
            ], layout=widgets.Layout(justify_content="space-around", width="70%")),
    ]))


def display_ship_placement():
    clear_output(wait=True)
    title, instructions = display_title_and_instructions()
    display(HTML("""
    <style>
        .grid-button {
            border: 1px solid black;
            width: 50px !important;
            height: 50px !important;
            padding: 0 !important;
        }
    </style>
    """))

    ship_list = [
        {"name": "Carrier", "size": 5},
        {"name": "Battleship", "size": 4},
        {"name": "Cruiser", "size": 3},
        {"name": "Submarine", "size": 3},
        {"name": "Destroyer", "size": 2},
    ]

    grid_buttons = create_grid_buttons_start()
    ship_buttons = create_ship_buttons(ship_list)

    # Create Start Game button
    start_game_button = widgets.Button(description="Start Game", button_style='success', layout=widgets.Layout(width='180px'))

    selected_ship = None
    ship_orientation = 'horizontal'
    placed_ships = set()

    def on_ship_button_click(button):
        nonlocal selected_ship
        if button.description not in placed_ships:
            selected_ship = button.description

    for button in ship_buttons:
        button.on_click(on_ship_button_click)

    def on_grid_button_click(button):
        nonlocal selected_ship, ship_orientation, placed_ships

        if not selected_ship:
            error_msg.value = '<p style="color: red; font-weight: bold; font-size: 20px;">Please select a ship first.</p>'
            return

        row, col = None, None
        for r, button_row in enumerate(grid_buttons):
            for c, b in enumerate(button_row):
                if b is button:
                    row, col = r, c
                    break

        ship_name, ship_size = selected_ship.split()[:2]
        ship_size = int(ship_size[1:-1])

        if ship_orientation == 'horizontal':
            if col + ship_size > 10:
                error_msg.value = '<p style="color: red; font-weight: bold; font-size: 20px;">Ship does not fit horizontally. Try another position.</p>'
                return

            for i in range(ship_size):
                if grid_buttons[row][col + i].description != "":
                    error_msg.value = '<p style="color: red; font-weight: bold; font-size: 20px;">There is already a ship in the way. Try another position.</p>'
                    return

            for i in range(ship_size):
                grid_buttons[row][col + i].description = "S"

        else:
            if row + ship_size > 10:
                error_msg.value = '<p style="color: red; font-weight: bold; font-size: 20px;">Ship does not fit vertically. Try another position.</p>'
                return

            for i in range(ship_size):
                if grid_buttons[row + i][col].description != "":
                    error_msg.value = '<p style="color: red; font-weight: bold; font-size: 20px;">There is already a ship in the way. Try another position.</p>'
                    return

            for i in range(ship_size):
                grid_buttons[row + i][col].description = "S"

        placed_ships.add(selected_ship)
        # After placed the ship disable it
        for button in ship_buttons:
          if button.description == selected_ship:
            button.disabled= True

        selected_ship = None
        progress_bar.value += 1
        # Check if all ships have been placed
        if progress_bar.value == len(ship_list):
            # Generate the computer's board
            computer_board = generate_computer_ships(ship_list)
            # Proceed to the next phase of the game
        error_msg.value = ""

        if progress_bar.value == len(ship_list):
            error_msg.value = '<p style="color: green; font-weight: bold; font-size: 20px;">All ships placed. You are ready to start the game!</p>'
            start_game_button.layout.visibility = 'visible'
            user_board = [[button.description for button in button_row] for button_row in grid_buttons]
            start_game_button.on_click(lambda _: start_game(user_board, computer_board))

    for row in grid_buttons:
        for button in row:
            button.on_click(on_grid_button_click)

    orientation_button = widgets.ToggleButtons(
        options=['horizontal', 'vertical'],
        description='Orientation:',
        disabled=False,
        button_style='',
    )

    def on_orientation_change(change):
        nonlocal ship_orientation
        ship_orientation = change['new']

    orientation_button.observe(on_orientation_change, names='value')

    progress_bar = widgets.FloatProgress(value=0, min=0, max=len(ship_list), description='Progress:')

    error_msg = widgets.HTML(value='')

    create_grid_and_ships_layout(grid_buttons, ship_buttons, title, instructions, orientation_button, progress_bar, error_msg, start_game_button)


output.register_callback('display_ship_placement', display_ship_placement)

In [47]:
import random

def generate_computer_ships(ship_list):
    computer_board = [['' for _ in range(10)] for _ in range(10)]
    for ship in ship_list:
        ship_placed = False
        while not ship_placed:
            row = random.randint(0, 9)
            col = random.randint(0, 9)
            orientation = random.choice(['horizontal', 'vertical'])

            if can_place_ship(computer_board, row, col, ship['size'], orientation):
                if orientation == 'horizontal':
                    for i in range(ship['size']):
                        computer_board[row][col + i] = "S"
                else:
                    for i in range(ship['size']):
                        computer_board[row + i][col] = "S"
                ship_placed = True
    return computer_board


def can_place_ship(board, row, col, ship_size, orientation):
    if orientation == 'horizontal':
        if col + ship_size > 10:
            return False
        for i in range(ship_size):
            if board[row][col + i] != '':
                return False
    else:
        if row + ship_size > 10:
            return False
        for i in range(ship_size):
            if board[row + i][col] != '':
                return False
    return True


In [48]:
def get_random_question():
    questions = FBconn.get('/questions', None)
    question_keys = list(questions.keys())
    random_key = random.choice(question_keys)
    return questions[random_key], random_key

In [49]:
def display_trivia_question(question_data, grid_buttons, user_board):
    clear_output(wait=True)
    # Title
    display(HTML("<h1 style='text-align: center;'>Cloud Battleship Trivia</h1>"))

    question_text = widgets.HTML(value="<h2 style='text-align: center;'>" + question_data['Question'] + "</h2>")

     # Button style
    button_style = widgets.ButtonStyle(
        button_color="lightblue",
        font_weight="bold",
        border_radius="12px"
    )

    answer_buttons = [
        widgets.Button(description=question_data[f"Option{i+1}"],button_style='info', layout=widgets.Layout(width="700px"))
        for i in range(4)
    ]

    for button in answer_buttons:
        button.on_click(lambda button_instance: on_answer_click(button_instance, question_data, grid_buttons, answer_buttons, user_board))

    answer_box = widgets.VBox([answer_buttons[0], answer_buttons[1],answer_buttons[2], answer_buttons[3]
    ], layout=widgets.Layout(justify_content='center'))

    question_box = widgets.VBox([question_text, answer_box, computer_attacking_label], layout=widgets.Layout(align_items='center', margin='0 0 20px 0'))
    display(question_box)
    return question_box

In [50]:
def on_answer_click(button, question_data, grid_buttons, answer_buttons, user_board):
    correct_answer = question_data['Option' + str(question_data['Correct'])]
    global correct_answers, incorrect_answers

    # Disable all answer buttons to prevent multiple answers.
    for b in answer_buttons:
        b.disabled = True

    if button.description == correct_answer:
        correct_answers = correct_answers + 1
        button.button_style = 'success'
        computer_attacking_label.value='<p style="color: green; font-weight: bold; font-size: 20px;">You can attack!</p>'
        enable_grid_buttons(grid_buttons)

    else:
        incorrect_answers = incorrect_answers + 1
        button.button_style = 'danger'
        computer_attacking_label.value='<p style="color: red; font-weight: bold; font-size: 20px;">Computer is attacking!... Please wait</p>'
        time.sleep(2.5)
        disable_grid_buttons(grid_buttons)
        computer_attack(user_board)

        if is_game_over(user_board):
          computer_attacking_label.value='<p style="color: blue; font-weight: bold; font-size: 20px;">You have been destroyed! You lost!</p>'
          time.sleep(2)
          computer_attacking_label.value=''
          save_stats_to_db("computer")
          display_end_game()
          return
        else:
          clear_output(wait=True)

        computer_attacking_label.value='<p style="color: red; font-weight: bold; font-size: 20px;">Get ready for the next question</p>'
        time.sleep(2.5)
        computer_attacking_label.value =""
        new_question_data, _ = get_random_question()
        display_trivia_question(new_question_data, grid_buttons, user_board)
        display_attack_grid(grid_buttons)
        create_menu_buttons(user_board)


In [51]:
def on_attack_click(computer_board, button, grid_buttons, user_board):
    button_row, button_col = button.row, button.col
    question_data, _ = get_random_question()
    if computer_board[button_row][button_col] == 'S':
        button.description = 'X'
        computer_board[button_row][button_col] = 'X'
        computer_attacking_label.value='<p style="color: green; font-weight: bold; font-size: 20px;">Hit!</p>'
        time.sleep(1.5)
        disable_grid_buttons(grid_buttons)
        button.style.button_color = 'red'
        if is_game_over(computer_board):
          computer_attacking_label.value='<p style="color: blue; font-weight: bold; font-size: 20px;">You have destroyed the enemy! Congratulation!</p>'
          time.sleep(2)
          save_stats_to_db("user")
          display_end_game()
        else:
          display_trivia_question(question_data, grid_buttons, user_board)
          display_attack_grid(grid_buttons)
    else:
        button.description = 'O'
        computer_board[button_row][button_col] = 'O'
        computer_attacking_label.value='<p style="color: red; font-weight: bold; font-size: 20px;">Miss!</p>'
        time.sleep(1.5)
        disable_grid_buttons(grid_buttons)
        button.style.button_color = 'lightgray'
        display_trivia_question(question_data, grid_buttons, user_board)
        display_attack_grid(grid_buttons)

    create_menu_buttons(user_board)
    computer_attacking_label.value =""

In [52]:
# This function will be called when the computer attacks.
def computer_attack(user_board):
    # Randomly select a target on the user's board.
    row = random.randint(0, len(user_board) - 1)
    col = random.randint(0, len(user_board[0]) - 1)

    # Check if the target cell has a ship ('S') or not ('').
    if user_board[row][col] == 'S':
        #print("The computer hit your ship!")
        # Update the cell on the user's board to indicate a hit.
        user_board[row][col] = 'X'
    else:
        #print("The computer missed!")
        # Update the cell on the user's board to indicate a miss.
        user_board[row][col] = 'O'

In [53]:
def is_game_over(board):
    # If there are no ships ('S') left on the board, the game is over.
    for row in board:
        if 'S' in row:
            return False  # Game is not over.
    return True  # Game is over.

In [54]:
def display_attack_grid(grid_buttons):
    grid_box = widgets.GridBox(
        [button for row in grid_buttons for button in row],
        layout=widgets.Layout(
            width='600px',
            height='600px',
            grid_template_columns='repeat(10, 50px)',
            grid_template_rows='repeat(10, 50px)',
            grid_gap='1px',
            justify_content='center'
        )
    )
    center_box = widgets.Box(
        [grid_box],
        layout=widgets.Layout(
            display='flex',
            justify_content='center',
            width='100%'
        )
    )

    disable_grid_buttons(grid_buttons)
    display(center_box)

In [55]:
def enable_grid_buttons(grid_buttons):
    for row in grid_buttons:
        for button in row:
          if button.description != 'X' or button.description != 'O':
            button.disabled = False

def disable_grid_buttons(grid_buttons):
    for row in grid_buttons:
        for button in row:
            button.disabled = True


In [56]:
# This function add sub-buttons to the game screen (exit, show stats, show my board)
def create_menu_buttons(user_board):

  # Button style
  button_style = widgets.ButtonStyle(
      button_color="lightblue",
      font_weight="bold",
      border_radius="12px"
  )

  # Create button widgets
  exit_button = widgets.Button(description="Exit Game", style=button_style, layout=widgets.Layout(width="200px"))
  stats_button = widgets.Button(description="Show Statistics",style=button_style, layout=widgets.Layout(width="200px"))
  board_button = widgets.Button(description="Show My Board",style=button_style, layout=widgets.Layout(width="200px"))

  # Arrange buttons in a box
  buttons_box = widgets.HBox([exit_button, stats_button, board_button])
  buttons_box.layout.justify_content = 'center'
  display(buttons_box)

  # Define button click events
  def on_exit_click(button):
      save_stats_to_db("user")
      display_end_game()

  def on_stats_click(button):
      # Insert code to show statistics here
      show_statistics()

  def on_board_click(button):
      # Insert code to show user board here
      display_user_grid(user_board)

  # Assign click events to buttons
  exit_button.on_click(on_exit_click)
  stats_button.on_click(on_stats_click)
  board_button.on_click(on_board_click)



In [57]:
def display_user_grid(user_board):
    # Create a new Output widget
    board_output = widgets.Output()
    display(board_output)

    with board_output:
        display(HTML("<h2 style='text-align: center;'>Your Board</h2>"))

        # Create the grid of buttons representing the user's board
        grid_buttons = [
            [
                widgets.Button(
                    description=cell if cell != "" else " ",
                    layout=widgets.Layout(width="50px", height="50px"),
                    style=widgets.ButtonStyle(button_color='lightblue'),
                    disabled=True,
                )
                for cell in row
            ]
            for row in user_board
        ]

        grid_box = widgets.GridBox(
            [button for row in grid_buttons for button in row],
            layout=widgets.Layout(
                width='600px',
                height='600px',
                grid_template_columns='repeat(10, 50px)',
                grid_template_rows='repeat(10, 50px)',
                grid_gap='1px',
                justify_content='center'
            )
        )
        center_box = widgets.Box(
            [grid_box],
            layout=widgets.Layout(
                display='flex',
                justify_content='center',
                width='100%'
            )
        )
        display(center_box)

        close_button = widgets.Button(description="Close")
        display(close_button)

        # Define what happens when the Close button is clicked
        def on_close_click(button):
            board_output.clear_output()

        close_button.on_click(on_close_click)

In [58]:
from IPython.display import display, Markdown, clear_output

def show_statistics():
    stats_output = widgets.Output()
    display(stats_output)
    with stats_output:
        display(HTML("<h2 style='text-align: center;'>Game Statistics</h2>"))
        display(HTML(f'<center>Correct answers: {correct_answers}</center>'))
        display(HTML(f'<center>Incorrect answers: {incorrect_answers}</center>'))

        if correct_answers + incorrect_answers > 0:
            labels = 'Correct', 'Incorrect'
            sizes = [correct_answers, incorrect_answers]
            colors = ['green', 'red']
            fig, ax = plt.subplots()
            ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
            ax.axis('equal')
            plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
            plt.show()
        else:
            display(HTML('<center>No questions were answered.</center>'))

        # Button style
        button_style = widgets.ButtonStyle(
            button_color="lightblue",
            font_weight="bold",
            border_radius="12px"
        )

        close_button = widgets.Button(description="Close",style=button_style, layout=widgets.Layout(width="200px"))
        display(close_button)

        def on_close_click(button):
            stats_output.clear_output()

        close_button.on_click(on_close_click)



In [59]:
def start_game(user_board, computer_board):
    clear_output(wait=True)

    # Title
    display(HTML("<h1 style='text-align: center;'>Cloud Battleship Trivia</h1>"))

    # Create attack grid buttons
    attack_grid_buttons = create_grid_buttons(computer_board, user_board)
    for row in (computer_board):
      print(row)
    time.sleep(3)

    # Trivia question and answers
    question_data, question_key = get_random_question()
    question_box = display_trivia_question(question_data, attack_grid_buttons, user_board)

    # Grid for attacking
    display_attack_grid(attack_grid_buttons)

    # Display menu buttons
    create_menu_buttons(user_board)



**End Game**

In [60]:
import matplotlib.pyplot as plt

# Function to display end game results and pie chart
def display_end_game():
    clear_output(wait=True)
    display(HTML("<h1 style='text-align: center; font-size: 50px;'>Game Over!</h1>"))
    display(HTML(f'<center>Correct answers: {correct_answers}</center>'))
    display(HTML(f'<center>Incorrect answers: {incorrect_answers}</center>'))

    if correct_answers + incorrect_answers > 0:
        labels = 'Correct', 'Incorrect'
        sizes = [correct_answers, incorrect_answers]
        colors = ['green', 'red']
        fig, ax = plt.subplots()
        ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        ax.axis('equal')
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
        plt.show()
    else:
        display(HTML('<center>No questions were answered.</center>'))

    # Button style
    button_style = widgets.ButtonStyle(
        button_color="lightblue",
        font_weight="bold",
        border_radius="12px"
    )

    menu_button = widgets.Button(description="Main Menu", style=button_style, layout=widgets.Layout(width="200px"))
    menu_button.layout.margin = "0 auto"
    display(widgets.HBox([menu_button], layout=widgets.Layout(justify_content='center')))

    def on_restart_click(button):
        global correct_answers, incorrect_answers
        clear_output(wait=True)
        correct_answers = 0
        incorrect_answers = 0
        display_main_menu()

    menu_button.on_click(on_restart_click)



In [61]:
# save stats to DB
def save_stats_to_db(winner):
  global correct_answers, incorrect_answers, current_user_id

  user_data = get_current_user()

  if winner == "user":
    total_wins = user_data["total_wins"] + 1
  elif winner == "computer":
    total_wins = user_data["total_wins"]

  # Calculate the new values for total_questions, and correct_answers
  total_questions = user_data["total_questions"] + correct_answers + incorrect_answers
  total_correct_answers = user_data["total_correct_answers"] + correct_answers

  # Update the user's data in Firebase
  FBconn.patch(f'/users/{current_user_id}', {"total_wins": total_wins,
                                              "total_questions": total_questions,
                                              "total_correct_answers": total_correct_answers})



def get_current_user():
    global current_user_id

    if current_user_id is None:
        raise Exception("No user is currently logged in")

    # Retrieve the user's data from Firebase
    user_data = FBconn.get('/users', current_user_id)

    if user_data is None:
        raise Exception(f"No user found with ID {current_user_id}")

    return user_data


**Leaderboard**

In [62]:
def display_leaderboard():
    clear_output(wait=True)

    html_code = """
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">

    <style>
    body {
      background-color: #f5f5f5;
    }

    h1 {
      font-family: 'Arial', sans-serif;
      color: #333;
      font-weight: 700;
      text-align: center;
    }

    .table {
      box-shadow: 0 0.75rem 1.5rem rgba(18,38,63,.03);
      border: 1px solid #e3e6f0;
      border-radius: .35rem;
    }

    .table thead th {
      padding: 1rem 1.5rem;
      font-size: 0.9rem;
      font-weight: 800;
      text-transform: uppercase;
      letter-spacing: .1em;
      color: #4e73df;
      border-top-width: 0;
      border-bottom-width: 2px;
      background-color: #f8f9fc;
    }

    .table tbody td {
      padding: 1rem;
      font-size: 1rem;
      color: #333;
      border-top-width: 1px;
    }

    .gold {
      background-color: #FFD700;
    }

    .silver {
      background-color: #C0C0C0;
    }

    .bronze {
      background-color: #CD7F32;
    }

    #backButton {
      background-color: #4e73df;
      border: none;
      color: white;
      padding: 15px 32px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 16px;
      margin: 4px 2px;
      cursor: pointer;
      border-radius: 12px;
    }

    </style>

    <div class="container d-flex justify-content-center align-items-center">
        <div class="text-center">
            <h1>Cloud Battleship Trivia Leaderboard</h1>
            <div id="leaderboard"></div>
            <button id="backButton" onclick="goBack()">Back to Main Page</button>
        </div>
    </div>

    <!-- JavaScript link for Bootstrap -->
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>

    <script>
        async function display_leaderboard() {
            var leaderboardResponse = await google.colab.kernel.invokeFunction("get_leaderboard", []);
            var leaderboardString = leaderboardResponse.data['text/plain'];
            leaderboardString = leaderboardString.substring(1, leaderboardString.length-1);
            leaderboardString = leaderboardString.replace(/'/g, '"');
            var leaderboard = JSON.parse(leaderboardString);

            var leaderboardHTML = `
                <div class="container mt-5">
                    <h1 class="mb-4">Leaderboard</h1>
                    <table class="table">
                        <thead>
                            <tr>
                                <th>Rank</th>
                                <th>Username</th>
                                <th>Total Wins</th>
                                <th>Total Correct Answers</th>
                                <th>Total Questions</th>
                            </tr>
                        </thead>
                        <tbody>
            `;

            leaderboard.forEach((user, index) => {
                let specialClass = "";
                if (index === 0) {
                    specialClass = "gold";
                } else if (index === 1) {
                    specialClass = "silver";
                } else if (index === 2) {
                    specialClass = "bronze";
                }

                leaderboardHTML += `
                    <tr class="${specialClass}">
                        <td>${index + 1}</td>
                        <td>${user.username}</td>
                        <td>${user.total_wins}</td>
                        <td>${user.total_correct_answers}</td>
                        <td>${user.total_questions}</td>
                    </tr>
                `;
            });

            leaderboardHTML += `
                        </tbody>
                    </table>
                </div>
            `;

            document.querySelector('#leaderboard').innerHTML = leaderboardHTML;
        }

        function goBack() {
            google.colab.kernel.invokeFunction("display_main_menu", []);
        }

        display_leaderboard();
    </script>
    """

    display(HTML(html_code))





In [63]:
import json
def get_leaderboard():
    users_data = FBconn.get('/users', None)

    # We need to convert the dictionary returned by Firebase to a list of dictionaries
    leaderboard = [{"username": k, **v} for k, v in users_data.items()]

    # Sort the leaderboard based on the specified criteria
    leaderboard = sorted(leaderboard, key=lambda user: (-user['total_wins'], -user['total_correct_answers'], user['username']))
    leaderboard = leaderboard[:10]

    # Return the top 10 users
    return json.dumps(leaderboard)



**Run this to start the game**

In [77]:
# Global Variables
computer_attacking_label = widgets.HTML(value='<p style="color: red; font-weight: bold; font-size: 20px;"></p>')
correct_answers = 0
incorrect_answers = 0
current_user_id = None

output.register_callback("check_login", check_login)
output.register_callback("user_exists", user_exists)
output.register_callback("register_user", register_user)
output.register_callback('display_rules', display_rules)
output.register_callback('display_admin_login', display_admin_login)
output.register_callback('display_leaderboard', display_leaderboard)
output.register_callback("get_leaderboard", get_leaderboard)
output.register_callback('display_main_menu', display_main_menu)
output.register_callback('display_login_menu', display_login_menu)
output.register_callback('on_submit_click', on_submit_click)
output.register_callback('on_back_click', on_back_click)
output.register_callback('on_add_question_click', on_add_question_click)
output.register_callback('on_edit_question_click', on_edit_question_click)
output.register_callback('get_current_user', get_current_user)

# Start the game
display_login_menu()